In [ ]:
import time
from adafruit_servokit import ServoKit

kit = ServoKit(channels=16)


mapp = {0:0,1:130,2:125}
# Reset all standard servos
for ch in [7,13,14,15]:
    try:
        kit.servo[ch].angle = mapp[ch]  # release (no signal)
    except Exception:
        pass

# Reset all continuous servos
for ch in range(16):
    try:
        kit.continuous_servo[ch].throttle = 0  # stop
        time.sleep(0)

        # or use None to release completely
    except Exception:
        pass

print("✅ All channels reset to neutral/off")


In [ ]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit

# -----------------------------
# PCA9685 setup
# -----------------------------
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c)
pca.frequency = 50  # standard servo frequency
kit = ServoKit(channels=16)  # For standard small servos

# -----------------------------
# Servo configuration
# -----------------------------
# Define which channels have large/high-torque vs small/RC servos
large_servos = [7]          # high-torque / metal gear
small_servos = [13, 14, 15] # standard hobby/RC servos

# Create servo objects for large servos with wide pulse range
large_servo_objects = {}
for ch in large_servos:
    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
    large_servo_objects[ch] = s
    s.angle = 0  # initialize to center
    time.sleep(0.05)

time.sleep(1)  # give all large servos time to reach position

# -----------------------------
# Helper functions
# -----------------------------
def move_to(servo_obj, current, target, step=2, delay=0.02):
    """
    Gradually move a large servo to the target angle
    """
    target = max(0, min(180, int(round(target))))
    if target == current:
        return current

    step = max(1, int(abs(step)))

    if target > current:
        angle = current
        while angle < target:
            angle = min(angle + step, target)
            servo_obj.angle = angle
            time.sleep(delay)
    else:
        angle = current
        while angle > target:
            angle = max(angle - step, target)
            servo_obj.angle = angle
            time.sleep(delay)

    return target  # updated current angle

def sweep_large_servo(ch):
    """
    Sweep a large servo on channel `ch` using gradual steps
    """
    s = large_servo_objects[ch]
    current_angle = 90
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 180, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)

def sweep_small_servo(ch):
    """
    Sweep a small hobby/RC servo using direct jumps
    """
    print(f"Sweeping small servo on channel {ch}")
    for angle in [0, 90, 180, 90, 0]:
        kit.servo[ch].angle = angle
        time.sleep(1)  # give servo time to reach position

# -----------------------------
# Main program
# -----------------------------
try:
    print("=== Sweeping LARGE servos ===")
    for ch in large_servos:
        print(f"Channel {ch}")
        sweep_large_servo(ch)

    print("\n=== Sweeping SMALL servos ===")
    for ch in small_servos:
        sweep_small_servo(ch)

finally:
    print("\nCleaning up PCA9685")
    pca.deinit()